# Binary Output in the LSTM

In [5]:
import matplotlib.pyplot as plt
import mlflow
import mlflow.keras
import numpy as np
from numpy import ndarray
import pandas as pd
from scipy.signal import medfilt
import sys

import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)

import tensorflow as tf

%load_ext lab_black

sys.path.append("/Users/renero/Documents/SideProjects/trader/src")
sys.path.append("/Users/renero/Documents/SideProjects/trader/src/predictor")
sys.path.append("/Users/renero/Documents/SideProjects/trader/src/utils")
sys.path.append("/Users/renero/Documents/SideProjects/trader/src/indicators")


from cs_dictionary import CSDictionary
from lstm import lstm_1layer, lstm_2layer
from sequences import sequences
from ticks import Ticks
from metrics import metrics

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


#### Read the data, scale and append indicators

In [2]:
argv = [
    "",
    "-c",
    "../src/predictor/params.dax.yaml",
    "-f",
    "../data/^GDAXI/dax_2018_2019.csv",
    "--window",
    "8",
    "--epochs",
    "200",
    "train",
]
params = CSDictionary(args=argv)

ticks = Ticks(params, params.input_file).scale()

ticks.append_indicator(["trend", "median_filter", "change"])
ticks.data.head()

2020-09-21 19:23:14 - INFO  - CSDictionary:__init__           - Using configuration parameters from: ../src/predictor/params.dax.yaml


,open,high,low,close,trend,median_filter,change,returns
Datetime,,,,,,,,
2018-01-02,0.71,0.66,0.61,0.67,1.0,0.67,0.00,0.000000
2018-01-03,0.73,0.78,0.77,0.79,1.0,0.79,0.12,0.179104
2018-01-04,0.90,0.99,0.96,1.00,1.0,1.00,0.21,0.265823
2018-01-05,1.07,1.13,1.13,1.17,1.0,1.17,0.17,0.170000
2018-01-08,1.26,1.21,1.26,1.22,1.0,1.17,0.05,0.042735


#### Split the dataset

In [4]:
X_train, y_train, X_test, y_test = ticks.prepare_for_training(
    predict="trend", train_columns=["close", "median_filter", "trend"]
)

#### Train the network and evaluate

In [4]:
def run(name, binary=True):
    nn = lstm_2layer(params, binary=binary)
    nn.start_training(X_train, y_train, name)
    nn.evaluate(X_test, y_test)
    nn.end_experiment()

#### Loop over different parameters

In [ ]:
for ws in [5, 10, 15, 20, 25, 30]:
    print(f"Window size = {ws}")
    params.window_size = ws
    X_train, y_train, X_test, y_test = ticks.prepare_for_training(
        predict="trend", train_columns=["close", "median_filter", "trend"]
    )
    run("trend_with_mf tw", binary=True)

In [8]:
params.window_size = 25
params.epochs = 1000
X_train, y_train, X_test, y_test = ticks.prepare_for_training(
    predict="trend", train_columns=["close", "median_filter", "trend"]
)
run("optimal ws and epoch nr.", binary=True)

2020-09-19 17:22:15 - INFO  - lstm_2layer:_build_model        - NN lstm_2layer created
2020-09-19 17:43:41 - INFO  - lstm_2layer:evaluate            - Predictions (yhat): (46,)
2020-09-19 17:43:41 - INFO  - lstm_2layer:evaluate            - Trend acc.: 0.61
